## Combining all downstream used A2TEA data into one RDS object
## Also performing necessary data manipulation

In [2]:
###packages currently installed via conda

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.14")

BiocManager::install("DESeq2")
#BiocManager::install("ggtree")
#BiocManager::install("Biostrings")

#lfcShrinkage isn't installable at the moment - problem for later
#install.packages("mvtnorm")
#BiocManager::install("apeglm")

#install.packages("data.table")

#install.packages("seqinr")
#install.packages("VennDiagram")
#install.packages("UpSetR")
#install.packages("cowplot")
#install.packages("ggplotify")

###############################################


if (!requireNamespace("BiocManager", quietly=TRUE))
    install.packages("BiocManager")

BiocManager::install("ggtree")
BiocManager::install("ggtreeExtra")
BiocManager::install("Biostrings")
BiocManager::install("survcomp")
BiocManager::install("seqinr")

install.packages("tidyverse")
install.packages("VennDiagram")
install.packages("UpSetR")
install.packages("cowplot")
install.packages("ggplotify")
install.packages("stringr")

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.3 (2022-03-10)

Old packages: 'uuid', 'vctrs'

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.3 (2022-03-10)

Installing package(s) 'DESeq2'

also installing the dependencies ‘sys’, ‘cachem’, ‘askpass’, ‘formatR’, ‘blob’, ‘memoise’, ‘plogr’, ‘png’, ‘curl’, ‘mime’, ‘openssl’, ‘colorspace’, ‘matrixStats’, ‘lambda.r’, ‘futile.options’, ‘DBI’, ‘RSQLite’, ‘KEGGREST’, ‘XML’, ‘xtable’, ‘httr’, ‘farver’, ‘labeling’, ‘munsell’, ‘viridisLite’, ‘MatrixGenerics’, ‘Matrix’, ‘DelayedArray’, ‘futile.logger’, ‘snow’, ‘BH’, ‘AnnotationDbi’, ‘annotate’, ‘survival’, ‘RColorBrewer’, ‘gtable’, ‘isoband’, ‘MASS’, ‘mgcv’, ‘s

In [16]:
library(DESeq2)
#library(BiocGenerics) # comes with DESeq2
#library(apeglm) # currently difficulties with install # better DESseq2 calc; later 

library(tidyverse)
library(ggtree)
library(Biostrings)
library(seqinr)
library(VennDiagram)
library(UpSetR)
library(cowplot)
library(ggplotify)
library(ape)
library(stringr)
library(survcomp)

#### Find all DESeq2 differential expression RDS objects and load them
##### Names are given based on the species/ecotype/etc. name

In [10]:
getwd()

[1] "/scratch2/cropbio/stoecker/A2TEA-pipeline/master_use3/a2tea/notebooks"

In [17]:
setwd("/scratch2/cropbio/stoecker/A2TEA-pipeline/master_use3/a2tea")

dea_list <- list.files(path = "R/deseq2/dea_final/", pattern = "dea_*", full.names=TRUE)
dea_list_short <- list.files(path = "R/deseq2/dea_final/", pattern = "dea_*", full.names=FALSE)

listRDS <- lapply(dea_list, readRDS, .GlobalEnv)

for (i in 1:length(listRDS)){
    assign(str_sub(dea_list_short[[i]], start=5), listRDS[[i]])
}

# remove unused combined list object
rm(listRDS)

In [ ]:
#nice for later - but needs install of package "apeglm"

#link on logFC shrinking
#https://support.bioconductor.org/p/77461/
#?lfcShrink
#lfcShrink(arabidopsis_dea, coef=2, type="apeglm")
#resultsNames(arabidopsis_dea)

In [ ]:
# DESeq2 testing
#At <- readRDS("R/deseq2/dea_final/dea_Arabidopsis_thaliana")
#DESeq2::results(At)
#as.data.frame(results(Arabidopsis_thaliana))

In [18]:
species_list <- vector()

for (i in 1:length(dea_list_short)){
    species_list <- c(species_list, str_sub(dea_list_short[[i]], start=5))
}

list_AllSpeciesDEResultsDataFrames <- list()

for (i in species_list) {
    # individual species file could be deleted but perhaps I'll need them later; kept for now...
    assign(paste0(i, "_DEresultsTable"), as.data.frame(results(get(i))))
    # create gene name vector here (when list is constructed rownames become unique species-gene combination)
    gene <- rownames(get(paste0(i, "_DEresultsTable")))
    df_with_genes <- add_column(get(paste0(i, "_DEresultsTable")), gene, .before = "baseMean")
    # also add a species column which will will also come in handy during the shiny steps
    species <- replicate(nrow(df_with_genes), i)
    df_with_genes <- add_column(df_with_genes, species, .before = "gene")
    # create list of dataframes, which will come in handy
    list_AllSpeciesDEResultsDataFrames[[i]] <- df_with_genes
    # list_AllSpeciesDEResultsDataFrames[[i]] <- (get(paste0(i, "_DEresultsTable")))
}

#str(list_AllSpeciesDEResultsDataFrames)

#create combined "mega" dataframe of all species, which is going  to be used for shiny lookup

combined_AllSpeciesDEResultsDataFrames <- do.call("rbind", list_AllSpeciesDEResultsDataFrames)


#str(combined_AllSpeciesDEResultsDataFrames)
#head(combined_AllSpeciesDEResultsDataFrames)
#head(list_AllSpeciesDEResultsDataFrames[[1]])
#?rbind

### creating long format HOG-genes relation table

In [19]:
HOG_file_raw <- read_delim("orthofinder/final-results/Phylogenetic_Hierarchical_Orthogroups/N0.tsv", delim = "\t")

#drop the OG, Gene Tree Parent Clade
HOG_file_short <- HOG_file_raw %>% select (-c(OG, `Gene Tree Parent Clade`)) 


#merge all species columns into one and remove the solo ones
#use all colnames except first - thus all species
HOG_file_merged <- HOG_file_short %>% unite(x,
                                          c(colnames(HOG_file_short)[-1]),
                                          sep = ",", 
                                          na.rm = TRUE,
                                          remove = TRUE)

#transform into long format - each row 1 gene and it's corresponding HOG in seperate columns
HOG_file_long <- HOG_file_merged %>%
                    mutate(unpacked = str_split(x, ",")) %>%
                    unnest(cols = c(unpacked)) %>%
                    mutate(genes = str_trim(unpacked)) %>% 
                    select(-c(x, unpacked)) %>% 
                    rename(gene = genes)

str(HOG_file_long)

Rows: 7656 Columns: 6
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (6): HOG, OG, Gene Tree Parent Clade, Arabidopsis_thaliana, Hordeum_vulg...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


tibble [54,126 × 2] (S3: tbl_df/tbl/data.frame)
 $ HOG : chr [1:54126] "N0.HOG0000000" "N0.HOG0000000" "N0.HOG0000000" "N0.HOG0000000" ...
 $ gene: chr [1:54126] "Zm00001d002239_P101" "Zm00001d002239_P098" "Zm00001d002239_P154" "Zm00001d002239_P036" ...


### adding specific HOG or singleton info as HOG column to DE tables

In [20]:
#?full_join
HOG_DE.a2tea <- full_join(combined_AllSpeciesDEResultsDataFrames, HOG_file_long, 
              by = c("gene")) %>% 
                replace_na(list(HOG = "singleton"))

#add column for significance - default level is padj <= 0.1

#subset tidyverse with the right functions ;D
#filter(HOG_DE.a2tea, padj < 0.05)
#?drop_na

significant <- vector()
for (FDR in HOG_DE.a2tea$padj) {
    if (!is.na(FDR) && FDR < 0.1) {
        significant <- c(significant, "yes")
    }
    else if (!is.na(FDR) && FDR > 0.1) {
        significant <- c(significant, "no")
    }
    else if (is.na(FDR)) {
        significant <- c(significant, "no")
    }
}

HOG_DE.a2tea <- add_column(HOG_DE.a2tea, significant, .after = "padj")

In [21]:
HOG_DE.a2tea %>% filter(HOG == "N0.HOG0002277") 

species,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,significant,HOG
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Hordeum_vulgare,HORVU2Hr1G076870.12,0.1572491,-0.7982638,4.080473,-0.1956302,0.8448996,NA,no,N0.HOG0002277
Hordeum_vulgare,HORVU2Hr1G076870.8,0.0000000,NA,NA,NA,NA,NA,no,N0.HOG0002277
Hordeum_vulgare,HORVU2Hr1G076870.10,0.0000000,NA,NA,NA,NA,NA,no,N0.HOG0002277
Hordeum_vulgare,HORVU2Hr1G076870.9,1.7534123,-3.0508510,3.599921,-0.8474771,0.3967292,NA,no,N0.HOG0002277
Hordeum_vulgare,HORVU2Hr1G076870.11,0.1571945,-0.7995849,4.080473,-0.1959540,0.8446462,NA,no,N0.HOG0002277
Hordeum_vulgare,HORVU2Hr1G076870.6,0.0000000,NA,NA,NA,NA,NA,no,N0.HOG0002277
Hordeum_vulgare,HORVU2Hr1G076870.7,0.4716760,-2.2772913,3.994710,-0.5700768,0.5686256,NA,no,N0.HOG0002277


### load in the rest of the data - hypotheses, trees, fasta and msa for the start 
### + toDo general stats, especially once Orthofinder calculates for HOG)
### +toDo Venn DIagrammes (until 5 species or so then alternative plots)

In [22]:
#extended_BLAST_hits <- readRDS("tea/2/extended_BLAST_hits/extended_BLAST_hits.RDS")
setClass("extended_BLAST_hits", 
         slots=list(blast_table="tbl_df",
                    num_genes_HOG="numeric",
                    num_genes_extend="numeric",
                    num_genes_complete="numeric",
                    genes_HOG="tbl_df",
                    genes_extend_hits="tbl_df")
         )
#expanded_OGs_short
#extended_BLAST_hits[["N0.HOG0000207"]]@nrow_table

In [ ]:
#readAAStringSet(paste0("tea/", 1, "/muscle/", "N0.HOG0000207", ".afa"))

In [18]:
#hypotheses

ERROR: Error in eval(expr, envir, enclos): object 'hypotheses' not found


### integrated
#### adding addtional OGs analysis output as modular attachments?
This needs to be done before the original steps since we add fasta, msa & trees to the sets of the add_OGs_analysis object first

In [23]:
## adding addtional OGs analysis output as modular attachments
#This needs to be done before the original steps since we add fasta, msa & trees 
#to the sets of the add_OGs_analysis object first before adding it as a list object during the original extended BLAST hits approach

setClass("add_OG_analysis",
         slots=list(add_OG_analysis="list")
         )

setClass("add_OG_set",
         slots=list(genes="tbl_df",
                #    fasta_files=read.fasta(paste0("tea/", hypothesis, "/fa_records/", exp_OG,".fa"), seqtype = "AA", as.string = TRUE), 
                    msa="AAStringSet", 
                    tree="phylo"
                   )
         )

for (h in hypotheses$hypothesis) {
    
  add_OG_analysis <- readRDS(paste0("tea/", h, "/add_OGs_object/add_OG_analysis_object.RDS"))


  for (n in names(add_OG_analysis)) {
      
    for (s in 1:length(add_OG_analysis[[n]]@add_OG_analysis)) {
      
      #adding msa info
      add_OG_analysis[[n]]@add_OG_analysis[[s]]@msa <- readAAStringSet(paste0("tea/", h, "/add_OGs_sets/muscle/", n, "/add_OGs_set_num-", s, ".afa.add"))
               
      #adding tree info
      add_OG_analysis[[n]]@add_OG_analysis[[s]]@tree <- read.tree(paste0("tea/", h, "/add_OGs_sets/trees/", n, "/add_OGs_set_num-", s, ".tree.add"))

    }
      
  }
}



### creating A2TEA object

In [24]:
hypotheses <- as.data.frame(
                t(
                  read.table("config/hypotheses.tsv",
                             header = FALSE,
                             sep = "\t",
                             row.names = NULL)
                )
              )
# first line as header/column names
names(hypotheses) <- hypotheses[1,]
# delete first line
hypotheses <- hypotheses[-1,]
# removal of row.names/numbering
row.names(hypotheses) <- NULL
#correct types
hypotheses$hypothesis <- as.numeric(hypotheses$hypothesis)
hypotheses$Nmin_expanded_in <- as.numeric(hypotheses$Nmin_expanded_in)
hypotheses$Nmin_compared_to <- as.numeric(hypotheses$Nmin_compared_to)
hypotheses$min_expansion_factor <- as.numeric(hypotheses$min_expansion_factor)



#create hypotheses object
# each object has list of exp. OGs
#iterate over list of hyptheses and associate expansions, fasta, msa and trees


#first goal - hypothesis object with hypothesis and all exp.HOGs

#could do it over objects in tea directory
#list.files(path = "tea", pattern = "*", full.names=FALSE)


#alternatively via parsing of hyptheses.tsv
hypotheses$hypothesis


## define three classes
# class for the expanded_OG - containing all different types of data we have on it
setClass("expanded_OG", slots=list(genes="spec_tbl_df",
                                   blast_table="tbl_df",
                                   num_genes_HOG="numeric",
                                   num_genes_extend="numeric",
                                   num_genes_complete="numeric",
                                   genes_HOG="tbl_df",
                                   genes_extend_hits="tbl_df",
#                                   fasta_files="list", 
                                   msa="AAStringSet", 
                                   tree="phylo",
                                   add_OG_analysis="list"))


# class for the hypotheses
# adding a prototype is essential here for ortho_intersect_plot since gg has to be defined as 
# register an old-style S3 class using setOldClass
#https://stackoverflow.com/questions/12636056/why-sometimes-i-cant-set-a-class-definition-as-slot-in-a-s4-class
#https://stackoverflow.com/questions/13841400/use-s3-virtual-class-as-slot-of-an-s4-class-got-error-got-class-s4-should-b
#had trouble getting this to work; thus solved differently
# -> seperate list object "Ortho_intersect_plots" hypothesis specific access via index
setClass("hypothesis", 
#         prototype=prototype(ortho_intersect_plot=structure(list(), class="gList")),
         slots=list(description="character", 
                                  number="character",
                                  expanded_in ="character", 
                                  compared_to="character", 
                                  expanded_OGs="list",
                                  species_tree="phylo"))
#                                  ortho_intersect_plot="gg"))

# class for extended BLAST hits info
setClass("extended_BLAST_hits", 
         slots=list(blast_table="tbl_df",
                    num_genes_HOG="numeric",
                    num_genes_extend="numeric",
                    num_genes_complete="numeric",
                    genes_HOG="tbl_df",
                    genes_extend_hits="tbl_df")
         )


#class for adding OGs analysis
setClass("add_OG_analysis",
         slots=list(add_OG_analysis="list")
         )
#another class for adding OGs analysis
setClass("add_OG_set",
         slots=list(genes="tbl_df",
                #    fasta_files=read.fasta(paste0("tea/", hypothesis, "/fa_records/", exp_OG,".fa"), seqtype = "AA", as.string = TRUE), 
                    msa="AAStringSet", 
                    tree="phylo"
                   )
         )


#rm(t)
#rm(hypothesis_1)
#rm(hypothesis_2)
#rm(HYPOTHESES.a2tea)



#remove protein_names in the snakemake pipeline - directories clean enough


for (hypothesis in hypotheses$hypothesis) {
    # read-in extended_BLAST_hits.RDS object of hypothesis
    extended_BLAST_hits <- readRDS(paste0("tea/", hypothesis, "/extended_BLAST_hits/extended_BLAST_hits.RDS"))
    
    # add_OG_analysis_object.RDS object of hypothesis
    add_OG_analysis <- readRDS(paste0("tea/", hypothesis, "/add_OGs_object/add_OG_analysis_object.RDS"))
    

    ## adding addtional OGs analysis output as modular attachments
    #This needs to be done before the original steps since we add fasta, msa & trees 
    #to the sets of the add_OGs_analysis object first before adding it as a list object 
    #during the original extended BLAST hits approach

    for (exp_OG in names(add_OG_analysis)) {
      
      for (set in 1:length(add_OG_analysis[[exp_OG]]@add_OG_analysis)) {
      
        #adding msa info
        add_OG_analysis[[exp_OG]]@add_OG_analysis[[set]]@msa <- readAAStringSet(paste0("tea/", hypothesis, "/add_OGs_sets/muscle/", exp_OG, "/add_OGs_set_num-", set, ".afa.add"))
               
        #adding tree info
        add_OG_analysis[[exp_OG]]@add_OG_analysis[[set]]@tree <- read.tree(paste0("tea/", hypothesis, "/add_OGs_sets/trees/", exp_OG, "/add_OGs_set_num-", set, ".tree.add"))

      }
      
    }
    
    
    # print(paste0("hypothesis_", hypo))
    
    # create empty list object for hypothesis #
    assign(paste0("hypothesis_", hypothesis), list())

    # assign list of names
    expanded_OGs <- list.files(path = paste0("tea/", hypothesis, "/exp_OGs_proteinnames/"), 
                               pattern = "*", 
                               full.names=FALSE)
    expanded_OGs_short <- str_sub(expanded_OGs, end=-5)

    # assign or subset the orthofinder species_tree based on the current hypothesis
    speciesTree <- ape::keep.tip(read.tree("orthofinder/final-results/Species_Tree/SpeciesTree_rooted_node_labels.txt"), 
                                 c(unlist(str_split(hypotheses$expanded_in[hypothesis], ";")), 
                                   unlist(str_split(hypotheses$compared_to[hypothesis], ";"))))
    
    
    #create empty list
    t <- list()

    for (exp_OG in expanded_OGs_short) {
    # print(exp_OG)
        test <- new("expanded_OG", 
             genes=read_table(paste0("tea/", hypothesis, "/exp_OGs_proteinnames/", exp_OG, ".txt"), col_names = FALSE),
             blast_table=extended_BLAST_hits[[exp_OG]]@blast_table,
             num_genes_HOG=extended_BLAST_hits[[exp_OG]]@num_genes_HOG,
             num_genes_extend=extended_BLAST_hits[[exp_OG]]@num_genes_extend,
             num_genes_complete=extended_BLAST_hits[[exp_OG]]@num_genes_complete,
             genes_HOG=extended_BLAST_hits[[exp_OG]]@genes_HOG,
             genes_extend_hits=extended_BLAST_hits[[exp_OG]]@genes_extend_hits,
#             fasta_files=read.fasta(paste0("tea/", hypothesis, "/fa_records/", exp_OG,".fa"), seqtype = "AA", as.string = TRUE), 
             msa=readAAStringSet(paste0("tea/", hypothesis, "/muscle/", exp_OG, ".afa")), 
             tree=read.tree(paste0("tea/", hypothesis, "/trees/", exp_OG, ".tree")),
             #here we add the complete add_OG analysis results (x sets per OG) to the original object
             add_OG_analysis=add_OG_analysis[[exp_OG]]@add_OG_analysis)
   
        x <- list(test)
        names(x) <- paste0(exp_OG)
        t <- c(t, x)
    }
 
    # create for each hypothesis a complete hypothesis object with correct naming
    # order: hypothesis@info (name, number, expand, compared, exp OGs)$indiv. exp_OGs@objects (exp.OG list, fa, msa, tree)
    # "expanded_in" and "compared_to" are split on ";" in case their size is > 1
    assign(paste0("hypothesis_", hypothesis),
              new("hypothesis", 
                  description=hypotheses$name[hypothesis],
                  number=as.character(hypothesis),
                  expanded_in=unlist(str_split(hypotheses$expanded_in[hypothesis], ";")),
                  compared_to=unlist(str_split(hypotheses$compared_to[hypothesis], ";")), 
                  expanded_OGs=t,
                  species_tree=speciesTree))#,
#                  ortho_intersect_plot=NULL))
}


# create empty list for final complete hypothesis object

#rm

# create name list of all hypotheses
hypotheses_list <- ls(pattern = "hypothesis_")

# create empty list object to completely hold all hypotheses and associated data
HYPOTHESES.a2tea <- list()

for (hypothesis in hypotheses_list) {
    h <- list(get(hypothesis))
    names(h) <- paste0(hypothesis)
    HYPOTHESES.a2tea <- c(HYPOTHESES.a2tea, h)
}

rm(hypothesis_1)
rm(hypothesis_2)


# final object is called:
# HYPOTHESES.a2tea


#add expanded OGs as column
#    add_column(get(paste0("hypothesis_", hypo)), expanded_OGs)
#class(hypothesis_1@expanded_OGs) <- c("expanded_OG", class(hypothesis_1@expanded_OGs))


[1] 1 2 3 4 5


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ───────────────────────────────────────────────────────────────────────


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ───────────────────────────────────────────────────────────────────────


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ───────────────────────────────────────────────────────────────────────


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ───────────────────────────────────────────────────────────────────────

...


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ───────────────────────────────────────────────────────────────────────


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ───────────────────────────────────────────────────────────────────────


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ───────────────────────────────────────────────────────────────────────


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ───────────────────────────────────────────────────────────────────────


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character()
)


── Column specification ───────────────────────────────────────────────────────────────────────

In [26]:
length(hypotheses$hypothesis)
#head(HYPOTHESES.a2tea$hypothesis_1@expanded_OGs$N0.HOG0000168@blast_table)
#head(HYPOTHESES.a2tea$hypothesis_2@expanded_OGs$N0.HOG0000028@blast_table)
HYPOTHESES.a2tea$hypothesis_2@expanded_OGs$N0.HOG0000028@add_OG_analysis$set_4@genes
HYPOTHESES.a2tea$hypothesis_2@expanded_OGs$N0.HOG0000028@add_OG_analysis$set_4@msa
HYPOTHESES.a2tea$hypothesis_2@expanded_OGs$N0.HOG0000028@add_OG_analysis$set_4@tree

[1] 5

value
<chr>
HORVU2Hr1G012140.2
HORVU2Hr1G012140.1
HORVU2Hr1G114100.9
HORVU2Hr1G114100.7
HORVU2Hr1G114100.10
HORVU2Hr1G114100.5
HORVU2Hr1G114100.6
HORVU2Hr1G114100.1
HORVU2Hr1G114100.3


AAStringSet object of length 63:
     width seq                                              names               
 [1]  1318 -----------------------...HEKIYVIVAVRCAILKRKFGKK Zm00001d006117_P0...
 [2]  1318 -----------------------...---------------------- Zm00001d006117_P0...
 [3]  1318 -----------------------...---------------------- Zm00001d006117_P0...
 [4]  1318 -----------------------...---------------------- Zm00001d006117_P0...
 [5]  1318 -----------------------...---------------------- Zm00001d006117_P0...
 ...   ... ...
[59]  1318 -----------------------...---------------------- HORVU2Hr1G114240....
[60]  1318 -----------------------...---------------------- HORVU2Hr1G114240....
[61]  1318 -----------------------...---------------------- HORVU2Hr1G114240....
[62]  1318 -----------------------...---------------------- HORVU2Hr1G114240....
[63]  1318 -----------------------...---------------------- HORVU2Hr1G114240....


Phylogenetic tree with 63 tips and 23 internal nodes.

Tip labels:
  HORVU2Hr1G114240.44, HORVU2Hr1G114240.46, HORVU2Hr1G114240.34, HORVU2Hr1G114240.7, HORVU2Hr1G114240.73, HORVU2Hr1G114240.74, ...
Node labels:
  , , 0.000, , 0.000, 0.000, ...

Unrooted; includes branch lengths.

### creating a non-redundant fasta file containing all genes part of the final objects 

In [58]:
n_hypotheses <- length(HYPOTHESES.a2tea)

A2TEA.fa.seqs <- vector(mode = "list")

for (hypothesis in 1:n_hypotheses) {
    
  exp_OGs <- names(HYPOTHESES.a2tea[[hypothesis]]@expanded_OGs)
    
  #extended hits analysis
  for (og in exp_OGs) {
    A2TEA.fa.seqs <- c(A2TEA.fa.seqs, read.fasta(paste0("tea/", 
                                hypothesis, 
                                "/fa_records/", 
                                og,
                                ".fa"), 
                         seqtype = "AA", 
                         as.string = TRUE))
  }  

  #add OGs analysis
  for (og in exp_OGs) {
    
    n_sets <- length(HYPOTHESES.a2tea[[hypothesis]]@expanded_OGs[[og]])
      
    for (set in 1:n_sets) {
        
      A2TEA.fa.seqs <- c(A2TEA.fa.seqs, read.fasta(paste0("tea/", 
                                hypothesis, 
                                "/add_OGs_sets/fa_records/", 
                                og,
                                "/add_OGs_set_num-", set, ".fa.add"), 
                         seqtype = "AA", 
                         as.string = TRUE))

    }
      
  }  
    
}


A2TEA.fa.seqs <- unique(A2TEA.fa.seqs)

#naming list elements - with id of gene/transcript/protein
for (i in 1:length(A2TEA.fa.seqs)) {
    
  names(A2TEA.fa.seqs)[i] <- attr(A2TEA.fa.seqs[[i]], which = "name")
}

object.size(A2TEA.fa.seqs)/1024/1024

45.2 bytes

In [59]:
#A2TEA.fa.seqs[["AT2G22980.2"]]

### testing size of fasta info

In [118]:
object.size(HYPOTHESES.a2tea)/1024/1024


#save(list = c("HYPOTHESES.a2tea"),file = "test_compress3", compress = "xz", compression_level = 9)

768.1 bytes

### HOG level tables -needs to be behind creation of hypotheses, because they are used here

In [23]:
# create hypothesis specfic HOG level file

#find all tibbles -list
#list.files(path = "tea/1/expansion_tibble", pattern = "expansion_tibble.rds", full.names=TRUE)


#length(hypotheses_list)


HOG_level_list <- list()

#for (i in 1:length(hypotheses_list)) {
for (i in 1:length(hypotheses$hypothesis)) {
    assign(paste0("hypothesis_", i, "_expansion_tibble"),
           readRDS(
               list.files(path = paste0("tea/", i, "/expansion_tibble"), 
                      pattern = "expansion_tibble.rds",
                      full.names=TRUE)
                  )
           )
    # add to list and name list entries according to hypothesis
    hog_level <- list(get(paste0("hypothesis_", i, "_expansion_tibble")))
    names(hog_level) <- paste0("hypothesis_", i)
    HOG_level_list <- c(HOG_level_list, hog_level)
}


#e.g. the following displays the tibble
#hypothesis_2_expansion_tibble

#r
#append "_gene_count" to all species in HOG tables
#get number of significantly regulated genes per HOG (per species and total) 
# -> (hmm perhaps do this once and then merge?)


# get count of species, HOG, significant combination from HOG_DE.a2tea
# reduce it to HOG, species, count
sig_genes_per_species_and_HOG <- HOG_DE.a2tea %>% 
                                     filter(significant == c("yes")) %>%
                                     filter(HOG != c("singleton")) %>%
                                        group_by(HOG, species, significant) %>% 
                                        mutate(count = n()) %>%
                                        ungroup() %>%
## don't use next line - bug!
#                                        select(HOG, species, count) %>%
            # https://www.r-bloggers.com/2018/05/workaround-for-tidyrspread-with-duplicate-row-identifiers/
            # spread error when no indexing for data
                                            group_by(species) %>% 
                                            mutate(grouped_id = row_number()) %>%  
                                            spread(species, count) %>% 
                                            select(-grouped_id) %>%
            # easy workaround for duplicated rows
                                                distinct()%>% 
            # rename species columns containing now the counts of sig. DE genes
                                                    rename_at(vars(-HOG), ~ paste0(., '_sigDE'))


#### SNIPPET caused some downstream problems - too many sig diff. genes
#    # and also add a column summing the rowwise sig. DE counts for all species
#    total_sigDE <- sig_genes_per_species_and_HOG %>% select(-HOG) %>% rowSums(na.rm = TRUE) 
#    sig_genes_per_species_and_HOG <- add_column(sig_genes_per_species_and_HOG, total_sigDE)
#    sig_genes_per_species_and_HOG <- sig_genes_per_species_and_HOG  %>%
#                                         group_by(HOG) %>%
#                                         # mutate all NAs to 0s
#                                         mutate_at(vars(-group_cols()), ~replace(., is.na(.), 0)) %>%
#                                         # merge rows per HOG - results in one line per HOG
#                                         #summarise_all(funs(sum))
#                                         summarise_all(list(sum))

#### switched to
#  h_expanded_in <- unlist(str_split(hypotheses$expanded_in[i], ";"))
#  h_compared_to <- unlist(str_split(hypotheses$compared_to[i], ";"))
#
#  h_species <- c(h_expanded_in, h_compared_to)
  #h_species

#  sig_genes_per_species_and_HOG <- sig_genes_per_species_and_HOG %>% 
#    mutate(
#      total_sigDE = rowSums(select(., paste0(h_species, "_sigDE")), na.rm=TRUE)  
#    )   

#  sig_genes_per_species_and_HOG <- sig_genes_per_species_and_HOG  %>%
#    group_by(HOG) %>%
#    # mutate all NAs to 0s#
#    mutate_at(vars(-group_cols()), ~replace(., is.na(.), 0)) %>%
#    # merge rows per HOG - results in one line per HOG
#    select(HOG, paste0(h_species, "_sigDE"), total_sigDE) %>%
#    distinct() %>%
#    summarise_all(list(sum))


# change the zeros back to NAs
#  sig_genes_per_species_and_HOG <- sig_genes_per_species_and_HOG  %>%
#                                         na_if(0)


#sig_genes_per_species_and_HOG
#total_sigDE
#inner_join(HOG_level_list[[1]], sig_genes_per_species_and_HOG, by = c("HOG"))
#full_join important - better to keep all HOG rows and allow filtering dynamically in the WebAPP itself

#head(sig_genes_per_species_and_HOG)

for (i in 1:length(HOG_level_list)) {
    
    h_expanded_in <- unlist(str_split(hypotheses$expanded_in[i], ";"))
    h_compared_to <- unlist(str_split(hypotheses$compared_to[i], ";"))
    h_species <- c(h_expanded_in, h_compared_to)
    #print(h_species)
    
    #print(head(sig_genes_per_species_and_HOG))
    #print("SPACER")
    
    #need to assign to a hypothesis specific name since this is a loop - duhh...
    h_sig_genes_per_species_and_HOG <- sig_genes_per_species_and_HOG %>% 
      mutate(
        total_sigDE = rowSums(select(., paste0(h_species, "_sigDE")), na.rm=TRUE)  
      )   

    h_sig_genes_per_species_and_HOG <- h_sig_genes_per_species_and_HOG  %>%
      group_by(HOG) %>%
      # mutate all NAs to 0s#
      mutate_at(vars(-group_cols()), ~replace(., is.na(.), 0)) %>%
      # merge rows per HOG - results in one line per HOG
      select(HOG, paste0(h_species, "_sigDE"), total_sigDE) %>%
      distinct() %>%
      summarise_all(list(sum))

    # change the zeros back to NAs
    h_sig_genes_per_species_and_HOG <- h_sig_genes_per_species_and_HOG  %>%
                                       na_if(0)
    
    HOG_level_list[[i]] <- HOG_level_list[[i]] %>% rename_at(vars(-HOG, -expansion), ~ paste0(., '_total'))
    HOG_level_list[[i]] <- full_join(HOG_level_list[[i]], h_sig_genes_per_species_and_HOG, by = c("HOG"))
}

str(HOG_level_list)

ERROR: [1m[33mError[39m in [1m[1m`mutate()`:[22m
[1m[22m[33m![39m Problem while computing `total_sigDE = rowSums(select(., paste0(h_species, "_sigDE")), na.rm = TRUE)`.
[1mCaused by error:[22m
[33m![39m error in evaluating the argument 'x' in selecting a method for function 'rowSums': Can't subset columns past the end.
[31m✖[39m Column `Zea_mays_sigDE` doesn't exist.


In [ ]:
HOG_level_list$hypothesis_2 %>%
  rowwise() %>%
  filter(Hordeum_vulgare_total < Hordeum_vulgare_sigDE) %>%
  head()

In [ ]:
library(stringr)
h_expanded_in <- unlist(str_split(hypotheses$expanded_in[1], ";"))
h_compared_to <- unlist(str_split(hypotheses$compared_to[1], ";"))

h_species <- c(h_expanded_in, h_compared_to)
h_species

# get count of species, HOG, significant combination from HOG_DE.a2tea
# reduce it to HOG, species, count
sig_genes_per_species_and_HOG <- HOG_DE.a2tea %>% 
                                     filter(significant == c("yes")) %>%
                                     filter(HOG != c("singleton")) %>%
                                        group_by(HOG, species, significant) %>% 
                                        mutate(count = n()) %>%
                                        ungroup() %>%



#select(HOG, species, count) %>%
            # https://www.r-bloggers.com/2018/05/workaround-for-tidyrspread-with-duplicate-row-identifiers/
            # spread error when no indexing for data
                                            group_by(species) %>% 
                                            mutate(grouped_id = row_number()) %>%  
                                            spread(species, count) %>%
                                            select(-grouped_id) %>%
            # easy workaround for duplicated rows
                                                distinct() %>% 
            # rename species columns containing now the counts of sig. DE genes
                                                    rename_at(vars(-HOG), ~ paste0(., '_sigDE'))

sig_genes_per_species_and_HOG %>% filter(HOG=="N0.HOG0002277")



sig_genes_per_species_and_HOG <- sig_genes_per_species_and_HOG %>% 
  mutate(
    total_sigDE = rowSums(select(., paste0(h_species, "_sigDE")), na.rm=TRUE)  
  ) 
      

    sig_genes_per_species_and_HOG <- sig_genes_per_species_and_HOG  %>%
                                         group_by(HOG) %>%
                                         # mutate all NAs to 0s#
                                         mutate_at(vars(-group_cols()), ~replace(., is.na(.), 0)) %>%
                                         # merge rows per HOG - results in one line per HOG
                                         select(HOG, paste0(h_species, "_sigDE"), total_sigDE) %>%
                                         distinct() %>%
                                         summarise_all(list(sum)) #%>%

sig_genes_per_species_and_HOG %>% filter(HOG=="N0.HOG0002277")



In [ ]:
summary(HYPOTHESES.a2tea)

In [ ]:
HOG_level_list$hypothesis_1
HOG_level_list$hypothesis_1 %>%
    filter(HOG == "N0.HOG0000318")

#HOG_level_list$hypothesis_1 %>% group_by(HOG) %>% summarise(sum(total_sigDE))

### checking for sigDE mistakes - where does this in the first row come from?

In [ ]:
#checking for sigDE mistakes - where does this in the first row come from?
HOG_level_list[[2]] %>%
  rowwise() %>%
  filter(Hordeum_vulgare_total < Hordeum_vulgare_sigDE) %>%
  head()

### Calculating enrichment of sig. diff. exp. genes of expanded_in species per HOG

In [ ]:
#install.packages("stringr")
library(stringr)

# what variables do I need?
#expanded_in = 
#compared_to =

#if no ";" (only one species)
#str_split(hh_expanded_in, ";")
#str_split(hh_compared_to, ";")

## respective length -> amount of species per category (since I need to divide/normalize by this)
#-> so n expanded_in species
#-> and n compared_to species

## use mutate to add new columns for normalized
#total number of sig. diff. genes - expanded_in species  /(divided by) n expanded_in species
#total number of sig. diff. genes - compared_to species  /(divided by) n compared_to species


#everything handled here
for (i in 1:length(hypotheses$hypothesis)) {
    h_expanded_in <- unlist(str_split(hypotheses$expanded_in[i], ";"))
    h_compared_to <- unlist(str_split(hypotheses$compared_to[i], ";"))
    length_expanded_in <- length(h_expanded_in)
    length_compared_to <- length(h_compared_to)
    
    HOG_level_list[[i]] <- HOG_level_list[[i]] %>%
      mutate(
        norm_sum_expanded_sigDE = rowSums(select(., paste0(h_expanded_in, "_sigDE")), na.rm=TRUE) #/ length_expanded_in
      )
    
    HOG_level_list[[i]] <- HOG_level_list[[i]] %>%
      mutate(
        norm_sum_compared_sigDE = rowSums(select(., paste0(h_compared_to, "_sigDE")), na.rm=TRUE) #/ length_compared_to
      )
      
    #compute column sums for norm_sum_expanded_sigDE and norm_sum_compared_sigDE
    #variable nuames correspond to the classic urn problem
    sum_white <- HOG_level_list[[i]] %>% 
      summarise(summed = sum(norm_sum_expanded_sigDE)) %>%
      pull()
    
    sum_black <- HOG_level_list[[i]] %>% 
      summarise(summed = sum(norm_sum_compared_sigDE)) %>%
      pull()
    
    
    #calculate 
    HOG_level_list[[i]] <- HOG_level_list[[i]] %>%
      rowwise() %>%
       mutate(
        oaes_hyper = phyper(norm_sum_expanded_sigDE - 1, 
                             sum_white, 
                             sum_black, 
                             norm_sum_expanded_sigDE + norm_sum_compared_sigDE, 
                             lower.tail = FALSE)
        ) %>% ungroup() %>%
       
      mutate(
        oaes_hyper = case_when(
                        oaes_hyper == 1 ~ NA_real_,
                        TRUE ~ oaes_hyper
                   )
      )

}
 

### OLD IDEAS
#total number of NON sig. diff. genes - expanded_in species  /(divided by) n expanded_in species
#total number of NON sig. diff. genes - compared_to species  /(divided by) n compared_to species

#finally a column of normalized total n genes per HOG
#-> sum of this column constitutes the normalized amount of all genes in all HOGs

#once we have these numbers we perform a multivariate hypergeomteric test
    
    #    HOG_level_list[[i]] <- HOG_level_list[[i]] %>%
#      mutate(
#        norm_sum_expanded_nonsig = (rowSums(select(., paste0(h_expanded_in, "_total")), na.rm=TRUE) / length_expanded_in) - (rowSums(select(., paste0(h_expanded_in, "_sigDE")), na.rm=TRUE) / length_expanded_in)
#      )
    
#    HOG_level_list[[i]] <- HOG_level_list[[i]] %>%
#      mutate(
#        norm_sum_compared_nonsig = (rowSums(select(., paste0(h_compared_to, "_total")), na.rm=TRUE) / length_compared_to) - (rowSums(select(., paste0(h_compared_to, "_sigDE")), na.rm=TRUE) / length_compared_to)
#      )

#    HOG_level_list[[i]] <- HOG_level_list[[i]] %>%
#      mutate(
#        norm_expanded_minus_compared_sigDE = norm_sum_expanded_sigDE - norm_sum_compared_sigDE
#      )
    
#    HOG_level_list[[i]] <- HOG_level_list[[i]] %>%
##      rowwise() %>%
#      mutate(
#        norm_sum_all_nonsig = rowSums(select(., "norm_sum_expanded_nonsig", "norm_sum_compared_nonsig"), na.rm=TRUE) / length_compared_to
#      )
    
    #perform phyper test - in cases where (expanded-compared) yields negative values will just default to 1
#    HOG_level_list[[i]] <- HOG_level_list[[i]] %>%
#      rowwise() %>%
#       mutate(
#        norm_phyper = phyper(
#                norm_expanded_minus_compared_sigDE, 
#                sum_white, 
#                sum_black, 
#                norm_sum_compared_nonsig, 
#                lower.tail = FALSE)
#         nom_phyper =  phyper(3, sum_white, sum_black, 5, lower.tail = FALSE)

#        )
    
#    HOG_level_list[[i]] <- HOG_level_list[[i]] %>%
#      mutate(
#        norm_sum = rowSums(
#                               select(.,
#                                      "norm_expanded_minus_compared_sigDE",  
#                                      "norm_sum_expanded_nonsig", 
#                                      "norm_sum_compared_nonsig"
#                                     ),
#                                na.rm=TRUE)
#      )

In [ ]:
#phyper exploration
a = 14
b = 11832
c = 1238
d = 0

e = a + d


phyper(a-1,b,c,e, lower.tail = FALSE)

### adding the computed cafe p-values to the HOG_level tables

In [ ]:
for (hypothesis_num in 1:length(HOG_level_list)) {

  cafe_tibble <- read_tsv(paste0("cafe/", hypothesis_num, "/cafe_complete_results/Gamma_family_results.txt"))[1:2]

  HOG_level_list[[hypothesis_num]] <- HOG_level_list[[hypothesis_num]] %>%
                                        left_join(., 
                                                  cafe_tibble, 
                                                  by = c("HOG" = "#FamilyID")
                                                 ) %>%
                                        rename(cafe_pvalue = pvalue) %>%
                                        relocate(cafe_pvalue, .after = oaes_hyper) %>%
                                        arrange(oaes_hyper)
}

In [ ]:
#cafe only outputs p-values until 0.001 - so I will as a quick hack change all 0s to 0.001
#perhaps I can modify the C++ code of CAFE5 a bit so I get longer doubles?
#on the other hand this is probably not really necessary
for (hypothesis_num in 1:length(HOG_level_list)) {

  HOG_level_list[[hypothesis_num]] <- HOG_level_list[[hypothesis_num]] %>%
    mutate(
      cafe_pvalue = case_when(
                      cafe_pvalue == 0 ~ 0.001,
                      TRUE ~ cafe_pvalue
                   )
    )
}

In [ ]:
HOG_level_list[[1]] %>% filter(expansion == "yes") %>% arrange(oaes_hyper) %>% head(n=20)
#HOG_level_list[[2]] %>% filter(expansion == "yes") %>% tail(n=20)

### combine overrep. analysis of sig. exp. genes with cafe value of gene expansion

In [ ]:
#install.packages("poolr")
#library(poolr)

#poolr::fisher(c(0.2, 0.05))
#>combined p-values with:      Fisher's method
#>number of p-values combined: 2 
#>test statistic:              9.21 ~ chi-square(df = 4) 
#>adjustment:                  none 
#>combined p-value:            0.0560517

#result can be accessed via $p

#poolr seems like the better suited package overall (alternatives to simple fisher's method),
#but sadly it's not installable via conda yet - thus kept here as a reminder
#also its manual has a neat short explanation of fisher's method

# for now survcomp's combine.test will have to make do
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")
#BiocManager::install("survcomp")
library(survcomp)

#combine.test(c(0.2, 0.05))
#>0.0560517018598809

#?combine.test
#typeof(
#combine.test(c(NA, NA), na.rm = TRUE)
#    )

In [ ]:
for (i in 1:length(hypotheses$hypothesis)) {
 
    #calculate the tea-value - trait-associated evolutionary adaption value
    HOG_level_list[[i]] <- HOG_level_list[[i]] %>%
      rowwise() %>%
      mutate(
        tea_value = case_when(
                      is.na(oaes_hyper) ~ NA_real_,
                      is.na(cafe_pvalue) ~ NA_real_,
                      is.na(oaes_hyper) & is.na(cafe_pvalue) ~ NA_real_,
                      TRUE ~ 1 # don't really get why I can't do it here:
                               # combine.test(c(oaes_hyper, cafe_pvalue), na.rm = TRUE)
                    )          # results in error as soon as both elements are NA
                               # seems like case_when calls everything anyways, even though the previous cases
                               # should make it skip the line..; anyways quick workaround with second mutate()
      ) %>%
      mutate(
        tea_value = case_when(
                      tea_value == 1 ~ combine.test(c(oaes_hyper, cafe_pvalue), na.rm = TRUE)
                    )
      ) %>%
      ungroup() %>%
      arrange(tea_value) 
}

In [ ]:
#HOG_level_list[[1]] %>% filter(expansion=="yes") %>% arrange(tea_value) %>% filter(tea_value <= 0.05)
#HOG_level_list[[1]] %>% select(tea_value) %>% pull() %>% na.omit() %>% hist()

In [ ]:

HOG_DE.a2tea %>% filter(HOG == "N0.HOG0000236")

### Create per hypothesis additional functional annotation table on HOG level 
#### (H/OG - non-redundant GO terms of all genes of all species in hypothesis)

In [ ]:
SFA <- readRDS("results/functional_annotation/species_functional_annotation.rds")

#we can use the HOG_file_long containing all genes - two columms: HOG; genes
#to add HOG info to all rows in the SFA tables
for (j in 1:length(SFA)) {
  #removal of all singletons? - for now no
  SFA[[j]] <- left_join(SFA[[j]], HOG_file_long, by = c("Protein-Accession" = "gene")) %>% 
    relocate(HOG, .before = "Protein-Accession") #%>%
#    filter(`HOG` != "NA")
  }


#"SFA" stands for Species Functional Annotation 
SFA_OG_level <- vector(mode = "list", length = length(HOG_level_list))

#create per hypothesis a HOG level GO annotation tibble that is non-redundant and free from unannotated HOGs
#add everything to the SFA_OG_level list object
for (i in 1:length(HOG_level_list)) {
    
    h_expanded_in <- unlist(str_split(hypotheses$expanded_in[i], ";"))
    h_compared_to <- unlist(str_split(hypotheses$compared_to[i], ";"))
    h_species <- unique(c(h_expanded_in, h_compared_to))
    

    #such an easy solution for combining df/tibbles that are list elements ;D
    #h_species is hypothesis specific, so we always pick the subset of species and genes that we need
    SFA_short <- bind_rows(SFA[h_species])

    #dummy encoding for all singletons/genes without a HOG
    #if we were to leave them as NA they are aggregated into one row
    SFA_short <- SFA_short %>%
      mutate(
      HOG = case_when(
        is.na(HOG) ~ paste0("singleton_rownum_", dplyr::row_number()),
        TRUE ~ HOG
      )
    )

  SFA_short <- SFA_short %>% 
    select(HOG, `Protein-Accession`, `Gene-Ontology-Term`) %>%
    group_by(HOG) %>%
    #remove redundancy while collapsing with ", "
    summarise(`Gene-Ontology-Term` = paste(unique(`Gene-Ontology-Term`), collapse=", ")) %>%
    #we should remove all rows that have NO GO terms associated
    #and remove all NAs from the Gene-Ontology-Term column if they have at least 1 GO term
    #remove inline NAs in Gene-Ontology-Term column
    mutate(
      `Gene-Ontology-Term` = str_remove_all(`Gene-Ontology-Term`, "NA, |, NA") 
    ) %>% 
    #remove lines with only NA in Gene-Ontology-Term column
    filter(!str_detect(`Gene-Ontology-Term`, 'NA'))

    
    #assign name of species to the ith element in list
    names(SFA_OG_level)[[i]] <- paste0("hypothesis_", i)
    #assign actual tibble with functional annotation of species to ith element of list
    SFA_OG_level[[i]] <- SFA_short
}



In [ ]:
head(SFA_OG_level[[1]])

### Last step: saving everything to one file which is input for the A2TEA WebApp

In [ ]:
save(hypotheses, 
     HYPOTHESES.a2tea, 
     HOG_DE.a2tea, 
     HOG_level_list, 
#     Ortho_intersect_plots,
     SFA,
     SFA_OG_level,
     file = "tea/A2TEA_finished.RData")

# To load the data again
#load("tea/A2TEA_finished.RData")

#perhaps use this to name output based on time?
#paste0("tea/A2TEA_finished_",
#    str_replace_all(
#    str_replace_all(
#        Sys.time(), ":", "-"),
#    " ", "_"
#                    ), ".RData")

In [ ]:
load("tea/A2TEA_finished.RData")

In [ ]:
HOG_level_list$hypothesis_2

In [ ]:
HYPOTHESES.a2tea$hypothesis_1@expanded_OGs$N0.HOG0000207@blast_table

### load optimization?

In [1]:
system.time(
    load("../tea/A2TEA_finished.RData")
    )

   user  system elapsed 
  9.267   0.346   9.619 

In [2]:
system.time(
    load("../../../better_calcs/a2tea/tea/A2TEA_finished.RData")
    )

   user  system elapsed 
  13.54    0.71   14.26 

In [14]:
A2TEA.fa.seqs[[1]]

[1] "MSLTLEFLLLLIVLILSHHAHSGSIVKFLPGFEGPLPFELETGYIGIGEEEEVQLFYYFIKSEKNPEEDPLLLWLSGGPGCSSLTGLLFENGPVALKFEVYNGSVPSLVSTTYSWTKMANIIFLDQPVGSGFSYSRTPLVDKISDTGEVKRIYEFLQKWLSKHQQFFSNPFYVGGDSYSGMIVPPLVQEIGKGNYQINLQGYILGNPITDTESEQNYQIPYAHGMALISDELYKSMERICKGNYVKVDSLNTKCYKLIKDYQKCIHKLNKYHILLPDCDITSPDCFLYRYTLITFWANNKSVREALQVNKGSIGKWVQCNYKNISYNYDIKSSVAYHMKNSIDGYRSLIYNGDHDMMVPFLATQAWIRSLNYSITDDWKPWMINDQIAGYTRSYSNKMTFATIKASLLVST"
attr(,"name")
[1] "AT2G22980.2"
attr(,"Annot")
[1] ">AT2G22980.2 pep chromosome:TAIR10:2:9778797:9783269:1 gene:AT2G22980 transcript:AT2G22980.2 gene_biotype:protein_coding transcript_biotype:protein_coding gene_symbol:SCPL13 description:Serine carboxypeptidase-like 13 [Source:UniProtKB/TrEMBL;Acc:A8MQS0]"
attr(,"class")
[1] "SeqFastaAA"

## Old ideas & approaches

### computation of tea value (first idea)

expansion ratio (exp/com) (9/4) x expression ratio (com/exp) (2/6) 
---- don't need (n_species_expanded/n_species_compared); stays equal in a hypothesis; similar to p-values not          comparable between different experiments
divided by HOG size (all genes in HOG for this particular hypothesis) x number of significantly regulated genes


problem - how to deal with missing values???

In [ ]:
# should I only have hypothesis species (total count and sigDE) HOG_level_list tables?

#new column tea-value
#if expansion yes continue; else NA value
#because we set expansion as criterium, species gene count has to be >=1
#sig DE counts can be NA ! - we could just add a pseudocount of 1 at these positions for the calculation

#so per line that has expansion = yes:
#get all expanded species; sum their gene counts; sum their sigDE counts
#get all compared species; sum their gene counts; sum their sigDE counts
#expansion_ratio <- divide summed gene count expanded by summed gene count compared
#expression_ratio <- divide summed SigDE count expanded by summed SigDE count compared
#e2_ratio <- divide expansion_ratio by expression_ratio
#tea-value <- divide e2_ratio by total number of genes in HOG and then by total number of SigDE in HOG


# outer most loop; going through hypotheses in HOG_level_list
for (i in 1:length(HOG_level_list)) {
    
    # access the names of all species important for the hypothesis - expanded_in; compared_to
    print(c(unlist(str_split(HYPOTHESES.a2tea[[i]]@expanded_in, ";")), 
            unlist(str_split(HYPOTHESES.a2tea[[i]]@compared_to, ";"))))
    
    # creating a relevant species vector might make this more future proof;
    # since I might change the completeness of the HOG_level_list elements
#    relevant_species <- c(unlist(str_split(HYPOTHESES.a2tea[[i]]@expanded_in, ";")), 
#                          unlist(str_split(HYPOTHESES.a2tea[[i]]@compared_to, ";")))
    
    expanded_species <- unlist(str_split(HYPOTHESES.a2tea[[i]]@expanded_in, ";"))
    
    compared_species <- unlist(str_split(HYPOTHESES.a2tea[[i]]@compared_to, ";"))                              
                                  
    ## pre-allocate empty vector for tea value with length of nrow of current hypothesis of HOG_level_list
    # neat; because NAs already in there so I can just skip if the HOG shows no expansion ;D
    tea_value <- rep(NA, nrow(HOG_level_list[[i]]))
    
    
    ## create "geneCount_workset" containing HOGs and counts of all relevant species
    geneCount_workset <-  HOG_level_list[[i]]

    # create geneCount_EXP_workset & geneCount_COMP_workset + drop columns that are unnecessary                  
    geneCount_EXP_workset <- geneCount_workset %>% 
                             select(ends_with("_total")) %>%
                             select(contains(expanded_species))
                        # also remove the "_total" ending
    geneCount_EXP_workset <- geneCount_EXP_workset %>% 
                             setNames(names(geneCount_EXP_workset) %>% 
                             stringr::str_replace("_total",""))
                          
    geneCount_COMP_workset <- geneCount_workset %>% 
                             select(ends_with("_total")) %>%
                             select(contains(compared_species))
                        # also remove the "_total" ending
    geneCount_COMP_workset <- geneCount_COMP_workset %>% 
                             setNames(names(geneCount_COMP_workset) %>% 
                             stringr::str_replace("_total",""))
                    

    ## create "geneSig_workset" containing HOGs and counts of all relevant species 
    geneSig_workset <-  HOG_level_list[[i]]
    
    # create geneSig_EXP_workset & geneSig_COMP_workset + drop columns that are unnecessary                  
    geneSig_EXP_workset <- geneSig_workset %>% 
                             select(ends_with("_sigDE")) %>%
                             select(contains(expanded_species))
                        # also remove the "_sigDE" ending
    geneSig_EXP_workset <- geneSig_EXP_workset %>% 
                             setNames(names(geneSig_EXP_workset) %>% 
                             stringr::str_replace("_sigDE",""))
                          
    geneSig_COMP_workset <- geneSig_workset %>% 
                             select(ends_with("_sigDE")) %>%
                             select(contains(compared_species))
                        # also remove the "_sigDE" ending
    geneSig_COMP_workset <- geneSig_COMP_workset %>% 
                             setNames(names(geneSig_COMP_workset) %>% 
                             stringr::str_replace("_sigDE",""))

                               
    # nested loop that goes through current hypothesis HOG_level_list line by line
    for (j in 1:nrow(HOG_level_list[[i]])) {
        # write value into index slot if current HOG does pass hard filter expansion criterium
        # if expansion == "no" do nothing; since we prefilled all positions with NA
        if (HOG_level_list[[i]][j,"expansion"] == "yes") {
            
            ## compute all the different sub-values for the tea_value
            # count of all genes in expanded species in HOG
            geneCount_EXP_HOG <- geneCount_EXP_workset[j,] %>%
                                 dplyr::rowwise() %>%
                                 sum()
            
            # count of all genes in compared species in HOG
            geneCount_COMP_HOG <- geneCount_COMP_workset[j,] %>%
                                      dplyr::rowwise() %>%
                                      sum()
            
            # sum genes of expanded & compared species in HOG
            geneCount_BOTH_HOG <- sum(geneCount_EXP_HOG, geneCount_COMP_HOG)
            
            
            # count of all sig DE genes in expanded species in HOG
            geneSig_EXP_HOG <- geneSig_EXP_workset[j,] %>%
                                 dplyr::rowwise() %>%
                                 sum()
            
            # count of all sig DE genes in compared species in HOG
            geneSig_COMP_HOG <- geneSig_COMP_workset[j,] %>%
                                      dplyr::rowwise() %>%
                                      sum()
            
            ## sigDE columns can contain zero (NA in table) counts
            ## we exchange this for a pseudocount 1
            ## doesn't impact tea_value too much
            if (is.na(geneSig_EXP_HOG)) {
                geneSig_EXP_HOG <- 1
            }
            
            if (is.na(geneSig_COMP_HOG)) {
                geneSig_COMP_HOG <- 1
            }
            
            # sum Sig DE genes of expanded & compared species in HOG
            geneSig_BOTH_HOG <- sum(geneSig_EXP_HOG, geneSig_COMP_HOG)

            
            ## perform tea value calculation
            int_tea_value <- (geneCount_EXP_HOG / geneCount_COMP_HOG) * (geneSig_COMP_HOG / geneSig_EXP_HOG) / 
                                                  (geneCount_BOTH_HOG * geneSig_BOTH_HOG)
            
            # lastly, add computed value to tea_value at index of current j loop position
            tea_value[j] <- int_tea_value
        }
    }
    
    ## add tea_value as new column to HOG_level_list table
    HOG_level_list[[i]] <- HOG_level_list[[i]] %>% 
                               add_column(tea_value, .after = "HOG")
    
}

### creating Venn (or alternative diagrammes for the hypothesis)
### -> will be moved to the WebApp and computed there...

In [ ]:
# create this awesome function to change area of plotting
# https://stackoverflow.com/questions/1279003/specify-width-and-height-of-plot
set_plot_dimensions <- function(width_choice, height_choice) {
        options(repr.plot.width=width_choice, repr.plot.height=height_choice)
        }
# use funtion to set area a bit wider
set_plot_dimensions(32, 10)



#step 1: removing all species not in hypothesis or keep only those that are
# using HOG_level_list and looking at the "_total" columns
#  only species in this hypothesis are in here ;D
# -> still drop all columns not necessary?
#step 2: perform check on size of this set
# -> if < 5 then create Venn diagramme
# -> if > 5 then create alternative plot
# create plot and safe as sub element in HYPOTHESES.a2tea object 



#HOG_DE.a2tea
#HOG_level_list[[1]]
#choose based on set length 2,3,4,5 after that UpsetR


Ortho_intersect_plots <- list()

for (i in 1:length(HOG_level_list)) {

workset <-  HOG_level_list[[i]]
# drop columns that are unnecessary
workset <- workset %>% select(HOG, ends_with("_total")) 
# add this point we can easily compute the number of species/ecotypes etc. 
# #species = (columns of current workset - 1) 
species_num <- ncol(workset) - 1
# change 0 to NAs since we can more easily filter on that
is.na(workset) <- workset==0
# remove '_total' from column names
#str_remove(colnames(workset), "_total")
workset <- workset %>% setNames(names(workset) %>% stringr::str_replace("_total",""))

# for testing purposes - remove Genotype vectors
rm(Genotype1,Genotype2,Genotype3)

genotypeList <- list()
for (j in 1:species_num) {
    assign(paste0("Genotype", j), as.vector(simplify_all(workset[!is.na(workset[j+1]),"HOG"])[[1]]))
    subList <- list(get(paste0("Genotype", j)))
    names(subList) <- colnames(workset)[j+1]
    genotypeList <- c(genotypeList, subList)
}


#using UpSetR
upsetOrtho <- upset(fromList(genotypeList), 
                   order.by = "freq",
                   empty.intersections = "on",
                   point.size = 3.5, 
                   line.size = 2, 
                   mainbar.y.label = "HOG Intersections", 
                   text.scale = c(2, 1.3, 1, 1, 2, 2))

p <- upsetOrtho

# if we have less than 5 species draw a venn diagramme
# combine both plots with some trickery
if (species_num < 5) {
    
# draw venn diagramme
orthoVenn <- venn.diagram(genotypeList,
                           print.mode = c("raw","percent"),
                           sigdigs = 1,
                           filename = NULL,
                           category = names(genotypeList),
                           fill = rainbow(length(names(genotypeList))),
                           cat.col = rainbow(length(names(genotypeList))),
                           cat.cex = 2,
                           cex = 1.5,
                           margin = 0.05
                           )    
    
# as.grob from ggplotify is necessary
orthoIntersect <- plot_grid(orthoVenn, 
          as.grob(upsetOrtho), 
          rel_widths = c(1/3, 2/3))

#p <- ggplotify::as.ggplot(orthoIntersect)
p <- orthoIntersect
    }

Ortho_intersect_plots[[i]] <- p

}
    
    
    


In [ ]:
Ortho_intersect_plots[[1]]